In [ ]:
%load_ext autoreload
%autoreload 2
import sike
import sike.post_processing
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sike.constants
import os

# Benchmarking: SIKE vs. FLYCHK vs. ADAS

This notebook compares the total radiated power, $\bar{L}_z$, and mean charge state, $\bar{z}$, predictions for impurity species in a background plasma at a range of $n_e$ and $T_e$. The definitions used here are:
- $\bar{L}_z = \frac{\sum_z L_z n_z}{\sum_z n_z}$, where $n_z$ is the density of charge state $z$ and $L_z = L_z^{ex} + L_z^{rr} + L_z^{br}$, which is the sum of radiative losses due to excitation, radiative recombination and Bremsstrahlung respectively. See refs [1], [2] and [3] for further details. 
- $\bar{z} = \frac{\sum_z{z n_z}}{\sum_z{n_z}}$

### Inputs

1. **SIKE**: See readme for installation instructions. Further details of SIKE can be found in [1].
2. **FLYCHK**: I used the values of $\bar{L}_z$, $\bar{Z}$ published online by the IAEA: https://www-amdis.iaea.org/FLYCHK/ZBAR/index.php. Clicking on each element brings up a page where tables of $\bar{L}_z$ and $\bar{Z}$ can be displayed, and these were copied into the .tsv files referenced in this notebook. 
3. **ADAS**: I used the radas package (https://github.com/cfs-energy/radas) to download data from open ADAS and put it into a .nc file to be read in by xarray.

### Differences between SIKE and FLYCHK
- It's not clear which atomic states go into FLYCHK rate calculations. The paper [2] suggests they use n_max=10, but there are 25 levels in the atomics.dat output files from running FLYCHK.
- SIKE does not include electron capture processes, and so cannot model dielectronic recombination. 
- SIKE does not include plasma effects unlike FLYCHK, which uses the ionic potential depression model [4].
- FLYCHK also includes the possibility of modelling plasmas with finite optical depth, although it's unclear if this is used in their published database. 

These limitations on SIKE apply on top of the limitations of FLYCHK, which are described here: https://www-amdis.iaea.org/FLYCHK/ZBAR/index.php

### References 
[1] Power, D. et al., pre-print: https://arxiv.org/abs/2410.00651

[2] Chung, H.-L. et al., HEDP 1 3-12 (2005)

[3] Summers, H. P. et al, Plasma Phys. Control. Fusion 48 263 (2006)

[4] Stewart & Pyatt, Astrophys. J. 144 1203 (1966)

# Compute comparisons

In [ ]:
# Define the velocity grid used in SIKE
E_min_ev = 1e-4
E_max_ev = 1e7
num_v = 500
vgrid, Egrid = sike.generate_vgrid(E_min_ev, E_max_ev, num_v)

In [ ]:
def compare_and_plot(
    el: str, j_resolved_sike_data: bool = False, savepath: str = "benchmarking/figures"
) -> None:
    """Read in or compute data from SIKE, FLYCHK and ADAS for a given element and plot the predicted mean charge state and radiated power vs. electron temperature

    :param el: Symbol of impurity species to process (e.g. "Li" or "Ne")
    :param j_resolved_sike_data: Whether to use the j-resolved atomic data from FAC in SIKE
    :param savepath: Directory to save figures
    """
    n_idxs = [
        0,
        2,
        4,
    ]  # Column numbers in FLYCHK tables defining the densities to compare against

    # Read in FLYCHK data
    nz_avg = np.loadtxt(
        "FLYCHK_data/" + el + "/nz_avg.tsv",
        delimiter="\t",
    )
    Te = nz_avg[1:, 0]
    ne = 1e6 * np.array([10 ** (float(str(n)[2:])) for n in nz_avg[0, 1:]])
    Zavg_flychk = nz_avg[1:, 1:]
    Lz_avg = np.loadtxt(
        "FLYCHK_data/" + el + "/Lz_avg.tsv",
        delimiter="\t",
    )
    Te = nz_avg[1:, 0]
    ne = 1e6 * np.array([10 ** (float(str(n)[2:])) for n in nz_avg[0, 1:]])
    Lzavg_tot_flychk = Lz_avg[1:, 1:]

    # Read in ADAS data
    el_full = sike.constants.SYMBOL2ELEMENT[el]
    adas_ds = xr.load_dataset(
        "/Users/power8/Documents/01_code/05_radas/radas/radas_dir/output/"
        + el_full
        + ".nc"
    )
    Zavg_adas = adas_ds.coronal_mean_charge_state.interp(
        dim_electron_density=ne, dim_electron_temp=Te
    ).values
    Lzavg_adas = (
        1e7
        * ne
        * (
            adas_ds.line_emission_from_excitation
            * adas_ds.coronal_charge_state_fraction
        )
        .sum(dim="dim_charge_state")
        .interp(dim_electron_density=ne, dim_electron_temp=Te)
        .values
    )
    Lzavg_rr_br_adas = (
        1e7
        * ne
        * (
            adas_ds.recombination_and_bremsstrahlung
            * adas_ds.coronal_charge_state_fraction
        )
        .sum(dim="dim_charge_state")
        .interp(dim_electron_density=ne, dim_electron_temp=Te)
        .values
    )
    Lzavg_tot_adas = (
        1e7
        * ne
        * adas_ds.coronal_Lz.interp(
            dim_electron_density=ne, dim_electron_temp=Te
        ).values
    )

    # Calculate SIKE data
    Te_sike = np.array([Te[i] for i in range(len(Te)) for _ in n_idxs])
    ne_sike = np.array([ne[i] for _ in range(len(Te)) for i in n_idxs])
    if j_resolved_sike_data:
        c = sike.SIKERun(
            Te=Te_sike,
            ne=ne_sike,
            element=el,
            vgrid=vgrid,
            saha_boltzmann_init=False,
            resolve_j=True,
            resolve_l=True,
        )
    else:
        c = sike.SIKERun(
            Te=Te_sike,
            ne=ne_sike,
            element=el,
            vgrid=vgrid,
            saha_boltzmann_init=False,
            resolve_j=False,
            resolve_l=False,
        )
    ds = c.evolve(dt_s=1e6)
    # ds = c.solve()
    Zavg_sike = sike.post_processing.get_Zavg(ds).values
    Lzavg_sike = 1e7 * 1e6 * ne_sike * sike.post_processing.get_Lz_avg(ds).values
    Lzavg_rr_sike = 1e7 * 1e6 * ne_sike * sike.post_processing.get_Lz_avg_rr(ds).values
    Lzavg_br_sike = 1e7 * 1e6 * ne_sike * sike.post_processing.get_Lz_avg_br(ds).values
    Zavg_sike.resize((len(Te), len(n_idxs)))
    Lzavg_sike.resize((len(Te), len(n_idxs)))
    Lzavg_rr_sike.resize((len(Te), len(n_idxs)))
    Lzavg_br_sike.resize((len(Te), len(n_idxs)))
    Lzavg_tot_sike = Lzavg_sike + Lzavg_rr_sike + Lzavg_br_sike

    # Plot mean charge
    fig, ax = plt.subplots(1)
    for i, i_n in enumerate(n_idxs):
        (l,) = ax.plot(
            Te,
            Zavg_flychk[:, i_n],
            label=r"$n_e=$" + "{:.0e}".format(ne[i_n]) + "m$^{-3}$",
        )
        ax.plot(Te, Zavg_sike[:, i], linestyle="--", color=l.get_color())
        ax.plot(Te, Zavg_adas[:, i_n], linestyle="-.", color=l.get_color())
    ax.set_xscale("log")
    ax.grid()
    ax.set_xlabel("$T_e$ [eV]")
    ax.set_ylabel(r"$\bar{Z}$")
    ax2 = ax.twinx()
    ax2.plot([], [], "-", color="black", label="FLYCHK")
    ax2.plot([], [], "--", color="black", label="SIKE")
    ax2.plot([], [], "-.", color="black", label="ADAS")
    ax2.set_yticks([])
    ax.legend(loc="upper left")
    ax2.legend(loc="lower right")
    ax.set_title(el)
    plt.show()
    fig.savefig(os.path.join(savepath, "z_" + el + ".pdf"))

    # Plot radiated power
    fig, ax = plt.subplots(1)
    for i, i_n in enumerate(n_idxs):
        (l,) = ax.plot(
            Te,
            Lzavg_tot_flychk[:, i_n],
            label=r"$n_e=$" + "{:.0e}".format(ne[i_n]) + "m$^{-3}$",
        )
        ax.plot(Te, Lzavg_tot_sike[:, i], linestyle="--", color=l.get_color())
        ax.plot(Te, Lzavg_tot_adas[:, i_n], linestyle="-.", color=l.get_color())
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.grid()
    ax.set_xlabel("$T_e$ [eV]")
    ax.set_ylabel(r"$n_e \bar{L}_z$ [ergs/s/atom]")
    ax2 = ax.twinx()
    ax2.plot([], [], "-", color="black", label="FLYCHK")
    ax2.plot([], [], "--", color="black", label="SIKE")
    ax2.plot([], [], "-.", color="black", label="ADAS")
    ax2.set_yticks([])
    ax.legend(loc="upper left")
    ax2.legend(loc="lower right")
    ax.set_title(el)
    plt.show()
    fig.savefig(os.path.join(savepath, "Lz_" + el + ".pdf"))

In [ ]:
compare_and_plot(
    "Ne",
    j_resolved_sike_data=False,
    savepath="figures/n_resolved_sike_data",
)